In [13]:
import os
from numpy import genfromtxt
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from scipy.signal import resample



## Dataloader:

In [14]:

# change this to wherever you store the dataset
dataset_dir = '/Users/liuxinqing/Documents/FallAllD'
os.chdir(dataset_dir)

oldDir=os.getcwd()
ParentDir=os.getcwd()
print(oldDir)
FileNamesAll=os.listdir(dataset_dir)
FileNames=[]
for f_name in FileNamesAll:
    if f_name.endswith('_A.dat'):
        FileNames.append(f_name)
LL=len(FileNames)

l_SubjectID=[]
l_Device=[]
l_ActivityID=[]
l_TrialNo=[]
l_Acc=[]
l_Gyr=[]
l_Mag=[]
l_Bar=[]



/Users/liuxinqing/Documents/FallAllD


In [15]:

for i in range(LL):
    f_name=FileNames[i]
    SubjectID=int(f_name[1:3])    
    l_SubjectID.append(np.uint8(SubjectID))
    ActivityID=int(f_name[8:11])    
    l_ActivityID.append(np.uint8(ActivityID))
    TrialNo=int(f_name[13:15])    
    l_TrialNo.append(np.uint8(TrialNo))
    Device=''
    if(int(f_name[5])==1):
        Device='Neck'
    else:
        if (int(f_name[5])==2):
            Device='Wrist'
        else:
            Device='Waist'    
    l_Device.append(Device)
    
    l_Acc.append(np.int16(genfromtxt(f_name, delimiter=',')))
    chArr=list(f_name)
    chArr[16]='G'
    f_name="".join(chArr)    
    l_Gyr.append(np.int16(genfromtxt(f_name, delimiter=',')))
    chArr=list(f_name)
    chArr[16]='M'
    f_name="".join(chArr)    
    l_Mag.append(np.int16(genfromtxt(f_name, delimiter=',')))
    chArr=list(f_name)
    chArr[16]='B'
    f_name="".join(chArr)    
    l_Bar.append(genfromtxt(f_name, delimiter=','))
    if i%100==0:
        print(f'File  {i+1}  out of {len(FileNames)}')
    #print(f'File  {i+1}  out of {len(FileNames)}')
os.chdir(oldDir)


File  1  out of 6605
File  101  out of 6605
File  201  out of 6605
File  301  out of 6605
File  401  out of 6605
File  501  out of 6605
File  601  out of 6605
File  701  out of 6605
File  801  out of 6605
File  901  out of 6605
File  1001  out of 6605
File  1101  out of 6605
File  1201  out of 6605
File  1301  out of 6605
File  1401  out of 6605
File  1501  out of 6605
File  1601  out of 6605
File  1701  out of 6605
File  1801  out of 6605
File  1901  out of 6605
File  2001  out of 6605
File  2101  out of 6605
File  2201  out of 6605
File  2301  out of 6605
File  2401  out of 6605
File  2501  out of 6605
File  2601  out of 6605
File  2701  out of 6605
File  2801  out of 6605
File  2901  out of 6605
File  3001  out of 6605
File  3101  out of 6605
File  3201  out of 6605
File  3301  out of 6605
File  3401  out of 6605
File  3501  out of 6605
File  3601  out of 6605
File  3701  out of 6605
File  3801  out of 6605
File  3901  out of 6605
File  4001  out of 6605
File  4101  out of 6605
File

In [33]:
FallAllD = pd.DataFrame(list(zip(l_SubjectID,l_Device,l_ActivityID,l_TrialNo,l_Acc,l_Gyr,l_Mag,l_Bar)), 
               columns =['SubjectID', 'Device','ActivityID','TrialNo','Acc','Gyr','Mag','Bar']) 

# .pkl and .h files are saved in the same directory as the dataset
FallAllD.to_pickle('FallAllD.pkl')
FallAllD.to_hdf('FallAllD.h5', key='df', mode='w')

/var/folders/6j/7v412jf92t1f972gcdmx7l8w0000gn/T/ipykernel_86256/2004310300.py:6: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Device', 'Acc', 'Gyr', 'Mag', 'Bar'], dtype='object')]

  FallAllD.to_hdf('FallAllD.h5', key='df', mode='w')


In [34]:

# change this to wherever you store the dataset
dataset_dir = '/Users/liuxinqing/Documents/FallAllD'
os.chdir(dataset_dir)

# Load the data
FallAllD = pd.read_pickle(f"{dataset_dir}/FallAllD.pkl")
# Print the first few rows of the DataFrame
#print(FallAllD.iloc[0])

print(len(FallAllD))

6605


In [35]:
fall_files=[]
not_fall_files=[]
#TODO: Splitting into Fall and Not Fall based on ActivtyID2Str.m file

# Define the range of ActivityIDs for fall and not fall
fall_ids = list(range(101, 136))
not_fall_ids = list(range(1, 45))

for index, row in FallAllD.iterrows():
    #print(row)
    # Extract ActivityID from row
    activity_id = int(row['ActivityID'])
    #print(activity_id)
    if activity_id in fall_ids:
        fall_files.append(row)
    elif activity_id in not_fall_ids:
        not_fall_files.append(row)


print("Fall files:", len(fall_files))
print("Not fall files:", len(not_fall_files))

Fall files: 1722
Not fall files: 4883


In [36]:
fall_files = sorted(fall_files, key=lambda x: x['ActivityID'])
not_fall_files = sorted(not_fall_files, key=lambda x: x['ActivityID'])
#print(fall_files)

In [37]:
def save_combined_data(
        input_files, 
        mode, 
        sample_rate=None,
        output_dir=None,
        label=None,
        ):
    '''
    mode: Acc / Acc+Gyr / Acc+Bar / Acc+Mag / Acc+Gyr+Bar
    '''

    #Acc (4760, 3)
    #Gyr (4760, 3)
    #Mag (1600, 3)
    #Bar (200, 2)


    acc_len = input_files[0]['Acc'].shape[0]
    duration = int(acc_len / 238)

    #print('duration', duration) #duration = 20  # 20 seconds
    # Calculate the new number of samples
    num_samples = sample_rate * duration # 40 * 20 = 800

    # Create a list for the output data
    output_data = []

    for i in range(len(input_files)):
        # Downsample the data for each sensor
        input_files[i]['Acc'] = resample(input_files[i]['Acc'], num_samples)
        input_files[i]['Gyr'] = resample(input_files[i]['Gyr'], num_samples)
        input_files[i]['Mag'] = resample(input_files[i]['Mag'], num_samples)
        input_files[i]['Bar'] = resample(input_files[i]['Bar'], num_samples)

        # Append the data to the output list based on the mode
        if mode == 'Acc':
            output_data.append(input_files[i]['Acc'])
        elif mode == 'Acc+Gyr':
            output_data.append(np.concatenate((input_files[i]['Acc'], input_files[i]['Gyr']), axis=1))
        elif mode == 'Acc+Bar':
            output_data.append(np.concatenate((input_files[i]['Acc'], input_files[i]['Bar']), axis=1))
        elif mode == 'Acc+Mag':
            output_data.append(np.concatenate((input_files[i]['Acc'], input_files[i]['Mag']), axis=1))
        elif mode == 'Acc+Gyr+Bar':
            output_data.append(np.concatenate((input_files[i]['Acc'], input_files[i]['Gyr'], input_files[i]['Bar']), axis=1))


    print('Acc', input_files[0]['Acc'].shape)
    print('Gyr', input_files[0]['Gyr'].shape)
    print('Mag', input_files[0]['Mag'].shape)
    print('Bar', input_files[0]['Bar'].shape)

    # Convert output_data to a numpy array
    output_data = np.array(output_data)
    print(output_data.shape) # (1722, 800, 8)


    # Save the data under the output_dir
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if label == 'fall':
        np.save(f'{output_dir}/fall.npy', output_data)
    elif label == 'not_fall':
        np.save(f'{output_dir}/not_fall.npy', output_data)
    #np.save(f'{output_dir}.npy', output_data)
    




In [38]:
output_dir = '/Users/liuxinqing/Documents/combined_data'
mode = 'Acc+Gyr+Bar'
sample_rate = 20

save_combined_data(fall_files, mode, sample_rate=sample_rate, output_dir=output_dir, label='fall')
save_combined_data(not_fall_files, mode, sample_rate=sample_rate, output_dir=output_dir, label='not_fall')

# Load the data from the .npy file
#loaded_data = np.load(f'{output_dir}/{mode}.npy')

#print(loaded_data.shape)

Acc (400, 3)
Gyr (400, 3)
Mag (400, 3)
Bar (400, 2)
(1722, 400, 8)
Acc (400, 3)
Gyr (400, 3)
Mag (400, 3)
Bar (400, 2)
(4883, 400, 8)
